In [1]:
from pathlib import Path
from kedro.framework.context import load_context

current_dir = Path.cwd()
proj_path = current_dir.parent
context = load_context(proj_path)

c:\users\valxa\anaconda3\envs\splatoon-ml\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
df_train = catalog.load("train")
df_test = catalog.load("test")
df_stage = catalog.load("stagedata")
df_weapon = catalog.load("weapon")


players = ["A1", "A2", "A3", "A4", "B1", "B2", "B3", "B4"]

2020-10-03 16:11:22,351 - kedro.io.data_catalog - INFO - Loading data from `train` (CSVDataSet)...
2020-10-03 16:11:22,542 - kedro.io.data_catalog - INFO - Loading data from `test` (CSVDataSet)...


c:\users\valxa\anaconda3\envs\splatoon-ml\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


2020-10-03 16:11:22,631 - kedro.io.data_catalog - INFO - Loading data from `stagedata` (CSVDataSet)...
2020-10-03 16:11:22,633 - kedro.io.data_catalog - INFO - Loading data from `weapon` (CSVDataSet)...


In [4]:
cat_cols = ['mode', 'stage', 'A1-weapon', 'A2-weapon','A3-weapon','A4-weapon',
          'B1-weapon', 'B2-weapon','B3-weapon','B4-weapon']
te_cat_cols = ['A1-mainweapon', 'A1-subweapon', 'A1-special', 'A1-category',
              'A2-mainweapon', 'A2-subweapon', 'A2-special', 'A2-category',
              'A3-mainweapon', 'A3-subweapon', 'A3-special', 'A3-category',
              'A4-mainweapon', 'A4-subweapon', 'A4-special', 'A4-category',
              'B1-mainweapon', 'B1-subweapon', 'B1-special', 'B1-category',
              'B2-mainweapon', 'B2-subweapon', 'B2-special', 'B2-category',
              'B3-mainweapon', 'B3-subweapon', 'B3-special', 'B3-category',
              'B4-mainweapon', 'B4-subweapon', 'B4-special', 'B4-category']


In [5]:
def disconnect_player_cnt(data):
    temp_data = data.copy()
    temp_data = temp_data.isnull()
    data["A-disconnect"] = temp_data["A1-weapon"].astype("int") + temp_data["A2-weapon"].astype("int") + temp_data["A3-weapon"].astype("int") + temp_data["A4-weapon"].astype("int")
    data["A-connect"] = 4 - data["A-disconnect"]
    data["B-disconnect"] = temp_data["B1-weapon"].astype("int") + temp_data["B2-weapon"].astype("int") + temp_data["B3-weapon"].astype("int") + temp_data["B4-weapon"].astype("int")
    data["B-connect"] = 4 - data["B-disconnect"]
    return data
    

In [6]:
def rank_to_value(x):
    rank = {'c-': 2, 'c': 3, 'c+': 4,
            'b-': 5, 'b': 6, 'b+': 7,
            'a-': 8, 'a': 9, 'a+': 10,
            's': 11, 's+': 12, 'x': 13
            }
    if x in rank:
        x = rank[x]
    else:
        x = 0
    return x

In [7]:
def regular_rank(data):
    players = ["A1", "A2", "A3", "A4", "B1", "B2", "B3", "B4"]
    for index, row in data.iterrows():
        if row["lobby-mode"] == "regular":
            for player in players:
                data.at[index, f"{player}-rank"] = 1
    return data


In [8]:
def rank_average(data):
    data["A-rank-ave"] = (data["A1-rank"] + data["A2-rank"] + data["A3-rank"] + data["A4-rank"]) / data["A-connect"]
    data["B-rank-ave"] = (data["B1-rank"] + data["B2-rank"] + data["B3-rank"] + data["B4-rank"]) / data["B-connect"]
    return data

In [9]:
def OneHot_encoder(x):
    import category_encoders as ce
    list_cols = ['lobby-mode', 'mode', 'stage']
    ce_ohe = ce.OneHotEncoder(cols=list_cols, handle_unknown='impute')
    return ce_ohe.fit_transform(x)
    

In [10]:
def target_encoder(train_x, train_y, test_x, cat_cols):
    from sklearn.model_selection import KFold
    
    for c in cat_cols:
        data_tmp = pd.DataFrame({c: train_x[c], 'target': train_y})
        target_mean = data_tmp.groupby(c)['target'].mean()
        
        test_x[c] = test_x[c].map(target_mean)
        
        tmp = np.repeat(np.nan, train_x.shape[0])
        
        kf = KFold(n_splits=4, shuffle=True, random_state=1234)
        
        for idx_1, idx_2 in kf.split(train_x):
            target_mean = data_tmp.iloc[idx_1].groupby(c)['target'].mean()
            tmp[idx_2] = train_x[c].iloc[idx_2].map(target_mean)
        train_x[c] = tmp
            

In [11]:
def level_average(data):
    data["A-level-ave"] = (data["A1-level"] + data["A2-level"] + data["A3-level"] + data["A4-level"]) / data["A-connect"]
    data["B-level-ave"] = (data["B1-level"] + data["B2-level"] + data["B3-level"] + data["B4-level"]) / data["B-connect"]
    return data

In [12]:
def data_augmentation(data):
    import random
    temp_data = data.copy()
    temp_data = temp_data.dropna()
    A_team = ["A1", "A2", "A3", "A4"]
    B_team = ["B1", "B2", "B3", "B4"]
    for index, row in temp_data.iterrows():
        rand_A = random.sample(A_team, 4)
        temp_data.at[index,"A1-weapon"], temp_data.at[index,"A2-weapon"], temp_data.at[index,"A3-weapon"], temp_data.at[index,"A4-weapon"] = \
            row[f"{rand_A[0]}-weapon"], row[f"{rand_A[1]}-weapon"], row[f"{rand_A[2]}-weapon"], row[f"{rand_A[3]}-weapon"]
        temp_data.at[index,"A1-rank"], temp_data.at[index,"A2-rank"], temp_data.at[index,"A3-rank"], temp_data.at[index,"A4-rank"] = \
            row[f"{rand_A[0]}-rank"], row[f"{rand_A[1]}-rank"], row[f"{rand_A[2]}-rank"], row[f"{rand_A[3]}-rank"]
        temp_data.at[index,"A1-level"], temp_data.at[index,"A2-level"], temp_data.at[index,"A3-level"], temp_data.at[index,"A4-level"] = \
            row[f"{rand_A[0]}-level"], row[f"{rand_A[1]}-level"], row[f"{rand_A[2]}-level"], row[f"{rand_A[3]}-level"]

        rand_B = random.sample(B_team, 4)
        temp_data.at[index,"B1-weapon"], temp_data.at[index,"B2-weapon"], temp_data.at[index,"B3-weapon"], temp_data.at[index,"B4-weapon"] = \
            row[f"{rand_B[0]}-weapon"], row[f"{rand_B[1]}-weapon"], row[f"{rand_B[2]}-weapon"], row[f"{rand_B[3]}-weapon"]
        temp_data.at[index,"B1-rank"], temp_data.at[index,"B2-rank"], temp_data.at[index,"B3-rank"], temp_data.at[index,"B4-rank"] = \
            row[f"{rand_B[0]}-rank"], row[f"{rand_B[1]}-rank"], row[f"{rand_B[2]}-rank"], row[f"{rand_B[3]}-rank"]
        temp_data.at[index,"B1-level"], temp_data.at[index,"B2-level"], temp_data.at[index,"B3-level"], temp_data.at[index,"B4-level"] = \
            row[f"{rand_B[0]}-level"], row[f"{rand_B[1]}-level"], row[f"{rand_B[2]}-level"], row[f"{rand_B[3]}-level"]
    data = pd.concat([data, temp_data])
    return data

In [13]:
def merge_weapon(data, weapondata):
    weapons = [p + "-weapon" for p in players]
    for weapon in weapons:
        temp_weapon_detail = data[[weapon]].join(weapondata.set_index("key"), on=weapon)
        weapon_detail = [weapon[:3] + col for col in temp_weapon_detail.columns]
        temp_weapon_detail.columns = weapon_detail
        data = pd.concat([data, temp_weapon_detail], axis=1)
        data = data.drop(weapon[:3] + weapon, axis=1)
        data = data.drop(weapon[:3] + 'i', axis=1)
        data = data.drop(weapon, axis=1)
    return data

In [14]:
def range_average(data):
    data["A-range-ave"] = (data["A1-range"] + data["A2-range"] + data["A3-range"] + data["A4-range"]) / data["A-connect"]
    data["B-range-ave"] = (data["B1-range"] + data["B2-range"] + data["B3-range"] + data["B4-range"]) / data["B-connect"]
    return data

In [15]:
def damage_average(data):
    data["A-damage-ave"] = (data["A1-damage"] + data["A2-damage"] + data["A3-damage"] + data["A4-damage"]) / data["A-connect"]
    data["B-damage-ave"] = (data["B1-damage"] + data["B2-damage"] + data["B3-damage"] + data["B4-damage"]) / data["B-connect"]
    return data

In [16]:
def rate_average(data):
    data["A-rate-ave"] = (data["A1-rate"] + data["A2-rate"] + data["A3-rate"] + data["A4-rate"]) / data["A-connect"]
    data["B-rate-ave"] = (data["B1-rate"] + data["B2-rate"] + data["B3-rate"] + data["B4-rate"]) / data["B-connect"]
    return data

In [17]:
def preprocess_weapon(weapon):
    weapon["category"] = weapon["category2"]
    weapon = weapon.drop(["category1", "category2"], axis=1)
    return weapon

In [18]:
def preprocess_stage(stagedata):
    return stagedata

In [19]:
def preprocess_data(train, test, weapon, stagedata):
    train = disconnect_player_cnt(train)
    train = data_augmentation(train)
    target = train['y']
    test = disconnect_player_cnt(test)
    data = pd.concat([train, test])
    
    # テーブルを結合
    players = ["A1", "A2", "A3", "A4", "B1", "B2", "B3", "B4"]
    data = data.join(stagedata.set_index("stage"), on="stage")
    data = merge_weapon(data, weapon)
    
    for player in players:
        data[f"{player}-rank"] = data[f"{player}-rank"].apply(rank_to_value)
        data[f"{player}-range"] = data[f"{player}-range"].fillna(0)
        data[f"{player}-damage"] = data[f"{player}-damage"].fillna(0)
        data[f"{player}-rate"] = data[f"{player}-rate"].fillna(0)
        data[f"{player}-level"] = data[f"{player}-level"].fillna(0)
    data = regular_rank(data)
    data = rank_average(data)
    data = level_average(data)
    data = range_average(data)
    data = damage_average(data)
    data = rate_average(data)
    data = OneHot_encoder(data)
    return data

In [20]:
target = data_augmentation(df_train)['y']

In [21]:
processed_weapon = preprocess_weapon(df_weapon)

c:\users\valxa\anaconda3\envs\splatoon-ml\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [22]:
processed_stage = preprocess_stage(df_stage)

In [23]:
processed_data = preprocess_data(df_train, df_test, processed_weapon, processed_stage)

c:\users\valxa\anaconda3\envs\splatoon-ml\lib\site-packages\patsy\constraint.py:13: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  from collections import Mapping
c:\users\valxa\anaconda3\envs\splatoon-ml\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [24]:
processed_data.columns

c:\users\valxa\anaconda3\envs\splatoon-ml\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Index(['id', 'period', 'game-ver', 'lobby-mode_1', 'lobby-mode_2', 'lobby',
       'mode_1', 'mode_2', 'mode_3', 'mode_4',
       ...
       'A-rank-ave', 'B-rank-ave', 'A-level-ave', 'B-level-ave', 'A-range-ave',
       'B-range-ave', 'A-damage-ave', 'B-damage-ave', 'A-rate-ave',
       'B-rate-ave'],
      dtype='object', length=122)

In [25]:
processed_data.head()

,id,period,game-ver,lobby-mode_1,lobby-mode_2,lobby,mode_1,mode_2,mode_3,mode_4,...,A-rank-ave,B-rank-ave,A-level-ave,B-level-ave,A-range-ave,B-range-ave,A-damage-ave,B-damage-ave,A-rate-ave,B-rate-ave
0,1,2019-10-15T20:00:00+00:00,5.0.1,1,0,standard,1,0,0,0,...,1.00,1.00,70.00,38.25,39.00,41.25,65.75,51.0,78.75,68.75
1,2,2019-12-14T04:00:00+00:00,5.0.1,1,0,standard,1,0,0,0,...,1.00,1.00,149.00,130.00,59.25,49.00,51.75,56.0,62.50,75.00
2,3,2019-12-25T14:00:00+00:00,5.0.1,0,1,standard,0,1,0,0,...,8.75,8.75,128.50,124.75,65.00,50.50,34.75,39.0,56.25,60.00
3,4,2019-11-11T14:00:00+00:00,5.0.1,1,0,standard,1,0,0,0,...,1.00,1.00,174.25,261.75,68.00,61.50,55.00,37.5,46.25,41.25
4,5,2019-12-14T06:00:00+00:00,5.0.1,0,1,standard,0,1,0,0,...,13.00,13.00,157.00,138.00,40.25,50.50,58.75,45.5,57.50,63.50


In [26]:
len(processed_data)

c:\users\valxa\anaconda3\envs\splatoon-ml\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


146048

In [27]:
def drop_column(data):
    drop_col = ["id", "period", "game-ver", "lobby",
               "A1-range", "A2-range", "A3-range", "A4-range", 
               "B1-range", "B2-range", "B3-range", "B4-range", 
               "A1-damage", "A2-damage", "A3-damage", "A4-damage", 
               "B1-damage", "B2-damage", "B3-damage", "B4-damage", 
               "A1-rate", "A2-rate", "A3-rate", "A4-rate", 
               "B1-rate", "B2-rate", "B3-rate", "B4-rate", ]
    data = data.drop(drop_col, axis=1)
    return data

In [28]:
def process_nan(data):
    data = data.fillna(0.0)
    return data

In [29]:
def create_master_table(data):
    master_table = drop_column(data)
    master_table = process_nan(master_table)
    train = master_table.iloc[:len(target), :]
    test = master_table.iloc[len(target):, :]
    test = test.drop('y', axis=1)
    train_x = train.drop('y', axis=1)
    train_y = train['y']
    target_encoder(train_x, train_y, test, te_cat_cols)
    train_x = process_nan(train_x)
    test = process_nan(test)
    return train_x, train_y, test

In [30]:
train_x, train_y, test = create_master_table(processed_data)

In [31]:
train_x.isnull().sum().sum()


0

In [32]:
train_x.to_csv("check_train_x.csv")

In [33]:
train_x.head()

,lobby-mode_1,lobby-mode_2,mode_1,mode_2,mode_3,mode_4,mode_5,stage_1,stage_2,stage_3,...,A-rank-ave,B-rank-ave,A-level-ave,B-level-ave,A-range-ave,B-range-ave,A-damage-ave,B-damage-ave,A-rate-ave,B-rate-ave
0,1,0,1,0,0,0,0,1,0,0,...,1.00,1.00,70.00,38.25,39.00,41.25,65.75,51.0,78.75,68.75
1,1,0,1,0,0,0,0,0,1,0,...,1.00,1.00,149.00,130.00,59.25,49.00,51.75,56.0,62.50,75.00
2,0,1,0,1,0,0,0,0,0,1,...,8.75,8.75,128.50,124.75,65.00,50.50,34.75,39.0,56.25,60.00
3,1,0,1,0,0,0,0,0,0,0,...,1.00,1.00,174.25,261.75,68.00,61.50,55.00,37.5,46.25,41.25
4,0,1,0,1,0,0,0,0,0,0,...,13.00,13.00,157.00,138.00,40.25,50.50,58.75,45.5,57.50,63.50


In [38]:
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold

In [39]:
params = {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'num_iterations': 1000,
    'early_stopping_rounds': 100,
}

In [40]:
FOLD_NUM = 10
kf = StratifiedKFold(n_splits=FOLD_NUM, shuffle=True, random_state=1234)

scores = []

pred_cv = np.zeros(len(test.index))

num_round = 1000

for i, (tdx, vdx) in enumerate(kf.split(train_x, train_y)):
    print(f'Fold : {i}')
    # 訓練用データと検証用データに分割
    X_train, X_valid, y_train, y_valid = train_x.iloc[tdx], train_x.iloc[vdx], train_y.values[tdx], train_y.values[vdx]
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_valid = lgb.Dataset(X_valid, y_valid)
    
    # 学習の実行
    model = lgb.train(params, lgb_train, num_boost_round=num_round,
                      valid_names=["train", "valid"], valid_sets=[lgb_train, lgb_valid],
                      verbose_eval=100)

    # 検証データに対する予測値を求めて、勝敗（０　or　１）に変換
    va_pred = np.round(model.predict(X_valid,num_iteration=model.best_iteration))
    
    # accuracyスコアを計算
    score_ = accuracy_score(y_valid, va_pred)
    
    # フォールド毎の検証時のスコアを格納
    scores.append(score_)
    
    #テストデータに対する予測値を求める
    submission = model.predict(test,num_iteration=model.best_iteration)
    
    #テストデータに対する予測値をフォールド数で割って蓄積
    #(フォールド毎の予測値の平均値を求めることと同じ)
    pred_cv += submission/FOLD_NUM

# 最終的なテストデータに対する予測値を勝敗（０　or　１）に変換
pred_cv = np.round(pred_cv)

# 最終的なaccuracyスコアを平均値で出力
print('')
print('################################')
print('CV_score:'+ str(np.mean(scores)))

Fold : 0


c:\users\valxa\anaconda3\envs\splatoon-ml\lib\site-packages\lightgbm\engine.py:151: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
c:\users\valxa\anaconda3\envs\splatoon-ml\lib\site-packages\lightgbm\engine.py:156: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Info] Number of positive: 55191, number of negative: 50746
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.011805 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6881
[LightGBM] [Info] Number of data points in the train set: 105937, number of used features: 93
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.520979 -> initscore=0.083967
[LightGBM] [Info] Start training from score 0.083967
Training until validation scores don't improve for 100 rounds
[100]	train's binary_logloss: 0.655142	valid's binary_logloss: 0.682761
[200]	train's binary_logloss: 0.630195	valid's binary_logloss: 0.681809
[300]	train's binary_logloss: 0.608131	valid's binary_logloss: 0.681083
[400]	train's binary_logloss: 0.586524	valid's binary_logloss: 0.679864
[500]	train's binary_logloss: 0.566793	valid's binary_logloss: 0.679186
[600]	train's binary_logloss: 0.547862	valid's binary_logloss: 0.677965
[700]	trai

[100]	train's binary_logloss: 0.655139	valid's binary_logloss: 0.68321
[200]	train's binary_logloss: 0.630006	valid's binary_logloss: 0.682121
Early stopping, best iteration is:
[195]	train's binary_logloss: 0.631278	valid's binary_logloss: 0.682059
Fold : 8
[LightGBM] [Info] Number of positive: 55192, number of negative: 50746
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.011603 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6877
[LightGBM] [Info] Number of data points in the train set: 105938, number of used features: 93
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.520984 -> initscore=0.083985
[LightGBM] [Info] Start training from score 0.083985
Training until validation scores don't improve for 100 rounds
[100]	train's binary_logloss: 0.655422	valid's binary_logloss: 0.68209
[200]	train's binary_logloss: 0.630745	valid's binary_logloss: 0.682125
Early stopping, best iteration is:
[169]

In [41]:
# 提出用ファイルを作成する
pd.DataFrame({"id": range(len(pred_cv)), "y": pred_cv }).to_csv("../data/07_model_output/submission.csv", index=False)

c:\users\valxa\anaconda3\envs\splatoon-ml\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
